In [ ]:
!pip install nlp_utils

In [ ]:
import nltk
import string
import pandas as pd
import nlp_utils as nu
import matplotlib.pyplot as plt
df=pd.read_csv('/content/dialogs.txt',sep='\t',names=['Question','Answer'])

In [ ]:
print(df)

                                               Question  \
0                                hi, how are you doing?   
1                         i'm fine. how about yourself?   
2                   i'm pretty good. thanks for asking.   
3                     no problem. so how have you been?   
4                      i've been great. what about you?   
...                                                 ...   
3720    that's a good question. maybe it's not old age.   
3721                              are you right-handed?   
3722                                  yes. all my life.   
3723  you're wearing out your right hand. stop using...   
3724        but i do all my writing with my right hand.   

                                                 Answer  
0                         i'm fine. how about yourself?  
1                   i'm pretty good. thanks for asking.  
2                     no problem. so how have you been?  
3                      i've been great. what about you?  
4

In [ ]:
# Removing special characters
import re
# importing regular expressions
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
# Lower case conversion
remove_n = lambda x: re.sub("\n", " ", x)
# removing \n and replacing them with empty value
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)
# removing non ascii characters
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
# removing alpha numeric values
df['Question'] = df['Question'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
# using map function and applying the function on query column
df['Answer'] = df['Answer'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
# using map function and applying the function on response column
print(df)
# final cleaned dataset

                                               Question  \
0                                hi  how are you doing    
1                         i m fine  how about yourself    
2                   i m pretty good  thanks for asking    
3                     no problem  so how have you been    
4                      i ve been great  what about you    
...                                                 ...   
3720    that s a good question  maybe it s not old age    
3721                              are you right handed    
3722                                  yes  all my life    
3723  you re wearing out your right hand  stop using...   
3724        but i do all my writing with my right hand    

                                                 Answer  
0                         i m fine  how about yourself   
1                   i m pretty good  thanks for asking   
2                     no problem  so how have you been   
3                      i ve been great  what about you   
4

In [ ]:
pip install transformers

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Load and preprocess data
X = df['Question']
y = df['Answer']

# Preprocess text data (tokenization, padding, etc.)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize and train a random forest classifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

from sklearn.ensemble import RandomForestClassifier

# Initialize and train the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)


# Initialize GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Fine-tune the GPT-2 model on your dataset

# Build and train an LSTM-based model
max_seq_length = 50
vocab_size = 10000

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_seq_length))
model_lstm.add(LSTM(128))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam')

# Train the LSTM model

# Implement ensemble techniques (e.g., stacking)
def ensemble_predictions(model1, model2, X):
    predictions1 = model1.predict(X)
    predictions2 = model2.predict(X)
    return (predictions1 + predictions2) / 2

# Create an ensemble of your models
ensemble_model = ensemble_predictions(rf_classifier, model_lstm, X_val)

# Evaluate the ensemble model
ensemble_predictions = ensemble_model > 0.5
ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")